<a href="https://colab.research.google.com/github/satwik-goyal/Robotics-CIFAR100-Classification/blob/main/CR_CWK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Keras

In [52]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Reshape , Multiply, Activation
from tensorflow.keras.layers import Input, DepthwiseConv2D, GlobalAveragePooling2D , Add, UpSampling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers.legacy import RMSprop, Adam
import matplotlib.pyplot as plt
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import ResNet50
from keras.models import Model
from keras.applications.resnet50 import preprocess_input, decode_predictions

In [53]:


 # Load CIFAR100 dataset
from tensorflow.keras.datasets import cifar100
(X_train, y_train), (X_test, y_test) = cifar100.load_data()

# # Normalize pixel values
# X_train = X_train.astype('float32') / 255
# X_test = X_test.astype('float32') / 255
x_train = preprocess_input(X_train)
x_test = preprocess_input(X_test)


# Convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, 100)
y_test = tf.keras.utils.to_categorical(y_test, 100)




In [ ]:
Selected_Image = 13422
image = x_train[Selected_Image]
plt.imshow(image)
plt.show()


## Pretrained

In [ ]:
import numpy as np


def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=224, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [ ]:
# data augmentation
from keras.preprocessing.image import ImageDataGenerator

train_data_generator = ImageDataGenerator(preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=True))

# Fit the data generator to your data
train_data_generator.fit(x_train)

# Apply the transformations to your data
# transformed_train_data = train_data_generator.flow(x_train, y_train, target_size=(224, 224),
#     batch_size=32,
#     class_mode='binary')

In [ ]:
# test_data_generator = ImageDataGenerator()

# transformed_test_data = test_data_generator.flow(X_test, y_test , batch_size = 32)

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False

model = Sequential()
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(.25))
model.add(BatchNormalization())
model.add(Dense(100, activation='softmax'))


optimizer = Adam(1e-2)
model.compile(optimizer = optimizer , metrics = ["accuracy"] , loss='categorical_crossentropy')


In [ ]:

model.fit_generator(train_data_generator.flow(x_train, y_train, batch_size=64),
                                  steps_per_epoch=x_train.shape[0] // 64,
                                  epochs=25,
                                  validation_data=(x_test, y_test))

In [9]:
history = pre_model.evaluate(X_test , y_test)

313/313 [==============================] - 5s 15ms/step - loss: 6.5759 - accuracy: 0.3407
